## Import modules

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

### 1. Increase / Descrease brightness of image

In [ ]:
def adjust_brightness(img_arr, scalar):
    img_res = img_arr + scalar
    return np.clip(img_res, 0, 255)

### 2. Adjust contrast of image

In [ ]:
def adjust_contrast(img_arr, scalar):
    img_res = img_arr * scalar
    return np.clip(img_res, 0, 255)

### 3. Flip image horizontally / vertically

In [ ]:
def flip(img_arr, direction):
    if direction == 'horizontal':
        return img_arr[:, ::-1]
    elif direction == 'vertical':
        return img_arr[::-1]

### 4. Convert to gray / sepia image

In [ ]:
def convert_gray_image(img_arr):
    img_res = np.copy(img_arr)
    if img_arr.shape[-1] == 4:
        img_res = np.delete(img_res, 3, axis=2)
    img_res = np.apply_along_axis(np.mean, 2, img_res)
    img_res = img_res.reshape(img_res.shape + (1,))
    return img_res

In [ ]:
def convert_sepia_image(img_arr):
    img_res = np.copy(img_arr)
    img_res[:, :, 0] = 0.393 * img_arr[:, :, 0] + 0.769 * img_arr[:, :, 1] + 0.189 * img_arr[:, :, 2]
    img_res[:, :, 1] = 0.349 * img_arr[:, :, 0] + 0.686 * img_arr[:, :, 1] + 0.168 * img_arr[:, :, 2]
    img_res[:, :, 2] = 0.272 * img_arr[:, :, 0] + 0.534 * img_arr[:, :, 1] + 0.131 * img_arr[:, :, 2]
    return np.clip(img_res, 0, 255)

### 5. Blur /Sharpen image

In [ ]:
def get_pixel(img_arr, i, j):
    n = img_arr.shape[0] 
    if i < 0 or j < 0 or i >= n or j >= n:
        return np.zeros(img_arr.shape[-1])
    return img_arr[i][j]

In [ ]:
def calculate_convo_pixel(img_arr, kernel, i, j):
    matrix = []
    for _i in range(i - 1, i + 2):
        for _j in range(j - 1, j + 2):
            matrix.append(get_pixel(img_arr, _i, _j))
    matrix = np.array(matrix).reshape((3, 3, img_arr.shape[-1]))
    kernel = kernel.reshape(kernel.shape + (1,))
    return np.sum(np.sum(matrix * kernel, axis=0), axis=0)

In [ ]:
def do_convolution(img_arr, kernel):
    n = img_arr.shape[0]
    img_res = np.zeros(img_arr.shape)
    for i in range(n):
        for j in range(n):
            img_res[i][j] = calculate_convo_pixel(img_arr, kernel, i, j)
    return img_res

In [ ]:
def blur(img_arr):
    kernel = np.array(1/16*np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]]))
    return np.clip(do_convolution(img_arr, kernel), 0, 255)

In [ ]:
def sharpen(img_arr):
    kernel = np.array(np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]))
    return np.clip((do_convolution(img_arr, kernel)), 0, 255)

### 6. Crop image to a specific size (center)

In [ ]:
def crop(img_arr, size=100):
    size = min(size, img_arr.shape[0]) // 2
    center = img_arr.shape[0] // 2
    img_res = img_arr[center-size:center+size,center-size:center+size]
    return img_res

### 7. Crop image with circle frame

In [ ]:
def calc_dist_to_center(r, i, j):
    return np.sqrt((i-r)**2 + (j-r)**2)

In [ ]:
def crop_circle_frame(img_arr):
    n = img_arr.shape[0]
    r = n // 2
    img_res = np.copy(img_arr)
    for i in range(n):
        j = 0
        while calc_dist_to_center(r, i, j) > r and j < n:
            img_res[i][j] = np.zeros(img_arr.shape[-1])
            if img_arr.shape[-1] == 4:
                img_res[i][j][-1] = 255
            j += 1
        j = n - 1
        while calc_dist_to_center(r, i, j) > r and j >= 0:
            img_res[i][j] = np.zeros(img_arr.shape[-1])
            if img_arr.shape[-1] == 4:
                img_res[i][j][-1] = 255
            j -= 1
    return img_res

### 8. Crop image with two crossovering ellipses 

In [ ]:
def find_focus(n):
    a = n * np.sqrt(2) / 2 - n / 10
    b = np.sqrt(n**2 / 2 - a**2)
    c = np.sqrt(a**2 - b**2)
    foci1 = np.round(n / 2 - c / np.sqrt(2))
    foci2 = n - 1 - foci1
    return np.array([foci1, foci2]), 2*a

In [ ]:
def sum_dist_to_focus(focus, i, j, main_diag):
    if(main_diag):
        return np.sum(
            np.sqrt((i - focus[0]) ** 2 + (j - focus[0]) ** 2)
            + np.sqrt((i - focus[1]) ** 2 + (j - focus[1]) ** 2)
        )
    else:
        return np.sum(
        np.sqrt((i - focus[0]) ** 2 + (j - focus[1]) ** 2)
        + np.sqrt((i - focus[1]) ** 2 + (j - focus[0]) ** 2)
    )

In [ ]:
def crop_ellipses_crossover(img_arr):
    n = img_arr.shape[0]
    focus, main_axis_len = find_focus(n)
    img_res = np.copy(img_arr)
    for i in range(n):
        j = 0
        while sum_dist_to_focus(focus, i, j, True) > main_axis_len and sum_dist_to_focus(focus, i, j, False) > main_axis_len and j < n:
            img_res[i][j] = np.zeros(img_arr.shape[-1])
            if img_arr.shape[-1] == 4:
                img_res[i][j][-1] = 255
            j+=1
        j = n // 2
        while sum_dist_to_focus(focus, i, j, True) > main_axis_len and sum_dist_to_focus(focus, i, j, False) > main_axis_len and j < n:
            img_res[i][j] = np.zeros(img_arr.shape[-1])
            if img_arr.shape[-1] == 4:
                img_res[i][j][-1] = 255
            j+=1
        j = n // 2
        while sum_dist_to_focus(focus, i, j, True) > main_axis_len and sum_dist_to_focus(focus, i, j, False) > main_axis_len and j >= 0:
            img_res[i][j] = np.zeros(img_arr.shape[-1])
            if img_arr.shape[-1] == 4:
                img_res[i][j][-1] = 255
            j -= 1
        j = n - 1
        while sum_dist_to_focus(focus, i, j, True) > main_axis_len and sum_dist_to_focus(focus, i, j, False) > main_axis_len and j >= 0:
            img_res[i][j] = np.zeros(img_arr.shape[-1])
            if img_arr.shape[-1] == 4:
                img_res[i][j][-1] = 255
            j -= 1
    return img_res

## Print menu for choosing a function 

In [ ]:
def print_menu_and_choose():
    print("""
    _____________________MENU________________________
    | 0. Do all functions below                     |
    | 1. Increase/Descrease brightness of image     |
    | 2. Adjust contrast of image                   |
    | 3. Flip image                                 |
    | 4. Convert to gray/sepia image                |
    | 5. Blur/Sharpen image                         |
    | 6. Crop image at center                       |
    | 7. Crop image with circle frame               |
    | 8. Crop image with two crossovering ellipses  |
    |_______________________________________________|
    """)
    return int(input("Please look at the menu in below ouput and Enter your choice between (0 - 8): "))

## Print and save image to output file

In [ ]:
def print_and_save_img(img_origin, img_name, img_output, choice, subchoice=0):
    choice -= 1
    img_output_postfix = [['brighten', 'darken'], ['contrast'], ['horizontal_flip', 'vertical_flip'], ['grayscale', 'sepia'], ['blur', 'sharpen'], ['crop_center'], ['crop_circle'], ['crop_ellipses']]
    img_name = img_name.split('.')
    # Check for grayscale image
    if img_output.shape[-1] == 1:
        img_output = img_output.reshape(img_output.shape[:2])
    # --------------- Print image -------------------
    _, axis = plt.subplots(1, 2, figsize=(8, 4))
    # Input image
    if img_origin.shape[-1] == 1:
        axis[0].set_title('Image_grayscale')
        axis[0].imshow(img_origin.astype('uint8'), cmap='gray', vmin=0, vmax=255)
    else: 
        axis[0].set_title('Image_origin')
        axis[0].imshow(img_origin.astype('uint8'))
    # Output image
    axis[1].set_title(f'Image_{img_output_postfix[choice][subchoice]}')
    if len(img_output.shape) == 2:
        axis[1].imshow(img_output.astype('uint8'), cmap="gray", vmin=0, vmax=255)
    else: 
        axis[1].imshow(img_output.astype('uint8'))
    # --------------- Save image -------------------
    Image.fromarray(img_output.astype('uint8')).save(f'{img_name[0]}_{img_output_postfix[choice][subchoice]}.{img_name[1]}')

## Main function 

In [ ]:
def main():
    # Open image and convert it into np array
    img_name = input('Name of img: ')
    img = Image.open(img_name)
    img_arr = np.array(img).astype("float")
    print("Image name: ", img_name)
    print("Image matrix shape: ", img_arr.shape)
    choice = print_menu_and_choose()
    """
    Processing img by predefined functions 
    *************************************
    """
    if choice == 1 or choice == 0:
        # 1. Increase and Descrease brightness of image
        img_brighten = adjust_brightness(img_arr, 50)
        img_darken = adjust_brightness(img_arr, -50)
        print_and_save_img(img_arr, img_name, img_brighten, 1)
        print_and_save_img(img_arr, img_name, img_darken, 1, 1)
    if choice == 2 or choice == 0:
        # 2. Adjust contrast of image
        img_contrast = adjust_contrast(img_arr, 1.5)
        print_and_save_img(img_arr, img_name, img_contrast, 2)
    if choice == 3 or choice == 0:
        # 3. Flip image
        opt = int(input("Enter 1 - horizonal or 2 - vertical: "))
        if(opt == 1):
            img_flip_hor = flip(img_arr, "horizontal")
            print_and_save_img(img_arr, img_name, img_flip_hor, 3)
        else: 
            img_flip_ver = flip(img_arr, "vertical")
            print_and_save_img(img_arr, img_name, img_flip_ver, 3, 1)
    if choice == 4 or choice == 0:
        # 4. Convert to gray / sepia image
        img_gray = convert_gray_image(img_arr)
        img_sepia = convert_sepia_image(img_arr)
        print_and_save_img(img_arr, img_name, img_gray, 4)
        print_and_save_img(img_arr, img_name, img_sepia, 4, 1)
    if choice == 5 or choice == 0:
        # 5. Blur / Sharpen image
        img_gray = convert_gray_image(img_arr)
        img_blur = blur(img_gray)
        img_sharpen = sharpen(img_gray)
        print_and_save_img(img_gray, img_name, img_blur, 5)
        print_and_save_img(img_blur, img_name, img_sharpen, 5, 1)
    if choice == 6 or choice == 0:
        # 6. Crop image center
        img_crop = crop(img_arr, 250)
        print_and_save_img(img_arr, img_name, img_crop, 6)
    if choice == 7 or choice == 0:
        # 7. Crop image with circle frame
        img_crop_circle = crop_circle_frame(img_arr)
        print_and_save_img(img_arr, img_name, img_crop_circle, 7)
    if choice == 8 or choice == 0:
        # 8. Crop image with two crossovering ellipses 
        img_crop_ellipses = crop_ellipses_crossover(img_arr)
        print_and_save_img(img_arr, img_name, img_crop_ellipses, 8)
    """ 
    *******************************************
    """
main()